## 3. A neutral-atom analog quantum processor

Now we are in a good shape to look at the features defining an analog neutral-atom quantum computer from a programer's perspective. Here is an example diagram that summarizes the main points:

<img src="./assets/register_ex.png" width="600" height="auto" />

Qubits are positioned in a 2D plane - the honeycomb lattice in this example. There are two main control knobs: (a) the geometric position of the qubits and (b) the time dependence of the lasers that control their energetics. While (b) comprises the traditional way we think about controlling qubits (pulses of some probe that interacts with qubits generating bitflips, phase gates, etc), the fact that the interaction between atoms depends on their relative distances means that (a) contributes equally to the dynamics of the quantum processor. As different dynamics correspond to different algorithms, both geometric and time-dependent controls are part of solving problems, or algorithm development. 

<img style="float: right;" src="./assets/MIS_ex.png" width="350" height="auto" />

The ability to control the quantum register geometry and inter-qubit connectivity leads to the concept of a "Field Programmable Qubit Array" (FPQA). In analogy with traditional embedded systems familiar to many electrical engineers, problem-dependent resource management and distribution allows for efficient problem solving. An example in combinatorial optimization is shown on the right, and we will learn more about this later in the course.

Geometry is important, but the time-dependent controls should not be overlooked. These are achieved by an in-plane light - technically composed by more than one laser - represented above by the green field behind the honeycomb register. By juggling with the intensity and phase of the field we are able to control the different parameters that control the qubit dynamics. These include the energy provided to the neutral-atoms to jump from $|g \rangle$ to Rydberg $|r\rangle$ states ($\Omega$), and energy shifts that bring qubits in or out of resonance with their bit-flipping frequencies, resulting on a qubit-dependent extra cost for excitation ($\Delta$).

In this chapter, we will cover the basic concepts regarding geometric (i) and time-dependent (ii) control of analog neutral-atom quantum computers.

Let's start by loading a few packages from Bloqade:
 

In [2]:
from bloqade import  piecewise_linear, piecewise_constant
from bloqade.atom_arrangement import ListOfLocations,  Square, Honeycomb
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a new tab
import numpy as np
output_notebook()

Loading BokehJS ...

### 3a. Setting up your own quantum processor


#### i) General atomic placement
Let's start setting up our first analog quantum computer register. Through this course, we will use QuEra's open package <a href="https://bloqade.quera.com/">Bloqade</a>. Bloqade allows us to seamlessly operate classical emulations as well as submit jobs to real quantum hardware.

To get started, let's start by declaring qubit positions by writing

In [3]:
my_register = ListOfLocations([(0.0, 0.0), (0.0, 5.0), (0.0, 9.0), (5.0, 2.0), (6.0, 7.0), (9.0, 10.0)])

Bloqade has visualization helpers for several of its functionalities. To take a peek at the register we just created, we do

In [4]:
my_register.show()

hovering your cursor over the atoms - the blue dots - will provide you with information regarding the physical position of choice and which index a given atom is represented by in Bloqade. Note that all positions are declared in $\mu m$!

We also see here a "blockade radius" toggle. This allows us to visualize and estimate connectivities, or interactions, between different qubits. The real importance of this tool will not come to play until later in the lectures.


In [ ]:
FreeResponseQuestion("exercise13ai")

#### ii) Regular atomic placement

For many applications, the positioning of qubits of interest follows a regular lattice pattern. Bloqade has shorthand lattice types for several useful lattices, all inside the `atom_arrangement` sub-module. These include the `Chain`, `Square`, `Rectangular`, `Honeycomb`, `Triangular`, `Lieb`, and `Kagome`. These objects are 'atom arrangements' right out of the box, so syntax is much simplified: call any of these geometries as methods, feeding them how many unit cells are desired in each primitive vector direction, and what is the lattice constant of choice (once more, to make sure we all remember, in $\mu m$).

Here goes an example for a uniform square lattice

In [5]:
Square(4,3,lattice_spacing=5.2).show()

While systems organized in lattices such as described above are interesting in their own right, they may lack some spice. To bring a degree of non-uniformity to this, we can randomly drop atoms out of the register. For example,

In [6]:
rng = np.random.default_rng(1234)
Honeycomb(3,3, lattice_spacing=4.5).apply_defect_density(0.3, rng=rng).remove_vacant_sites().show()

Here we have a honeycomb lattice with 3 unit cells in each of its principal directions and 30% of the qubits dropped out. 

The Bloqade code above hints at some subtle points of operating quantum hardware. When sending a command to Aquila asking for this register, Aquila generates lasers on all corresponding possible spots of the honeycomb lattice. But applying a defect density says that, when sorting qubits into place, some laser spots must be left empty. To "free" the lasers corresponding to the spots empty of qubits, we include the method `remove_vacant_sites()`, which ends up boosting the performance of the quantum computer, if a problem is to be submitted for real hardware.

#### iii) Geometric hardware constraints

When chosing positions for qubits in a register, we should be aware of the physical constraints imposed by real hardware. Whenever we feed a list of tuples into a `listOfLocations()`, we should remember that this command will go down a pipeline all the way until it reaches controls over laser beams that will position the atoms as desired. These controls, naturally, have finite precision. Furthermore, laser tweezers holding atoms cannot be brought too close to each other, as beams becomes indistinguishable to control cameras and atoms may collide. All of these are part of the physical constraints the physical hardware enforces on the users when design choices on their quantum registers.

The diagram below illustrates, with 9 qubits, the key geometric constraints the user must be aware of.

<img src="./assets/geometries.png" width="600" height="auto" />

In sum, these are:

- The qubit register must lie in a 2D plane.
- The register must lie in a "user region" of $75 \mu m \times 76 \mu m$.
- Qubits cannot be positioned closer than $4 \mu m$  apart, radially.
- Qubits can be slid continuously horizontally, but must be positioned along well-defined rows vertically spaced at least $4\mu m$ apart.
- The precision for positioning qubits in a given point in the user region is of $0.1 \mu m$.

These constraints come with various best practices. For instance, it is advisable to prioritize the vertical direction when designing elongated registers. Additionally, whenever feasible, consider producing well-spaced duplicates of your register within the user region to parallelize calculations.

In [ ]:
FreeResponseQuestion("exercise13aiii")

### 3b. Quantum dynamics

#### i) Physics of the Algorithm (Time evolution)

Any computing algorithm refers to the transformation of information as function of time, starting from an initially controlled state and arriving at a desired final state that corresponds to the answer of a given problem.

A quantum algorithm is no different. With this mindset, information is encoded in a quantum state $|\Psi\rangle$, whose time evolution is controlled by the laws of quantum mechanics via the Schrödinger equation

$$
i \hbar \frac{\partial}{\partial t} |\Psi\rangle = \mathcal{H}(t) |\Psi\rangle
$$

Here, $\hbar$ is the reduced Planck's constant, $i$ is the imaginary number. 

The specific form of this equation for our system is shown below. This equation states that $\mathcal{H}$, the Hamiltonian operator, modifies the state $|\Psi\rangle$ as time passes. On a digital quantum computer, the Hamiltonian is controlled in a discrete way, with pulses in time that generate quantum logic operations in a qubit. At that point, one may forget the explicit Schrödinger equation and follow the evolution of the state step by step. For analog machines, in contrast, Schrödinger's equation is the favored picture, as programming is achieved by smoothly controlling $\mathcal{H}$. 

Analog quantum computing programs tend to be heuristic. Algorithm development thus requires intimate understanding of a system's dynamic and, to achieve that, requires clear knowledge of the system Hamiltonian and how it can be manipulated.

#### ii) The Rydberg-interaction Hamiltonian

The energetics and time evolution of our analog quantum processor is controlled by the following Hamiltonian

$$
\frac{\mathcal{H}(t)}{\hbar} = \sum_i \frac{\Omega(t)}{2} \left( e^{i \phi(t) } | g_i \rangle  \langle r_i | + e^{-i \phi(t) } | r_i \rangle  \langle g_i | \right) - \sum_i \Delta(t) \hat{n}_i + \sum_{i < j} V_{ij} \hat{n}_i \hat{n}_j.
$$

In this expression, $| g_i \rangle$ or $| r_i \rangle$ correspond to the two possible states of qubit $i$ - that is, our computational basis - and $\hat{n}_i = | r_i \rangle \langle r_i |$ counts excitations to the Rydberg state. Each time-dependent term in the Hamiltonian is under the dynamic control of the user, and we call these time-varying controls *waveforms*. The last term $V_{ij}$, in contrast, can also be manipulated by the user but only geometrically.

Lets look at each term in the Hamiltonian individually.

The first term:


$ \sum_i \frac{\Omega(t)}{2} \left( e^{i \phi(t) } | g_i \rangle  \langle r_i | + e^{-i \phi(t) } | r_i \rangle  \langle g_i | \right)  $

contains two time-dependent parameters: $\Omega$, known as the "Rabi amplitude", and $\phi$, which is a complex phase. As can be seen, this Hamiltonian term is off-diagonal in the computational basis, and is responsible for qubit flips. For isolated qubits, a drive with a finite constant $\Omega$ would lead to Rabi oscillations, hence its name. This is also closely related with $X$ gates in gate-based quantum computing. The phase $\phi$ enables changing the rotation axis in the Bloch sphere, thus enabling the exchange of $X$-like rotations to $Y$-like rotations.

The next term:

$\sum_i \Delta(t) \hat{n}_i$

is controled one waveform $\Delta$, which we call the "detuning". Since, writing more explicitly,

$\hat{n}_i= 0|g_i \rangle  \langle g_i | + 1| r_i \rangle  \langle r_i |, $  

this term is diagonal in the computational basis. Thus, higher or lower values of $\Delta$ enable directly favoring or disfavoring the occupation of Rydberg states for a given qubit $i$. 

Finally, we can look at the last term in the Hamiltonian:

$\sum_{i < j} V_{ij} \hat{n}_i \hat{n}_j.$

This is a very important term, as it controls the interactions between different qubits. Without it, there can be no entanglement. This is known as a 2-body-term, as it depends on qubits $i$ and $j$ pairwise. The energy scale reads

$V_{ij}=\frac{C_6}{d_{ij}^6},$

where $C_6/2\pi=862,690 MHz \mu m^6$ is a constant and $d_{ij}$ is the Euclidean distance between qubits $i$ and $j$ on the plane.

This interaction imparts a positive energy cost to the Hamiltonian, and thus a repulsive interaction between qubits. Since $V_{ij}$, decays with the distance between the qubits to the 6th power, qubits that are too close can contribute unbearably high-energies to the system if they happen to be both excited at once (i.e. when $\hat{n}_i \hat{n}_j = 1 \times 1 = 1$). This is the source for entanglement in the quantum computing dynamics, and allows for conditional excitations (and thus is somewhat related to controlled logic quantum gates, like $CZ$ gates, in gate-based quantum computing).


#### iii) Declaring waveforms

To generate waveforms on Bloqade, the most basic form involves creating `piecewise_linear()` functions. While Bloqade can also generate smooth waveforms, piecewise-linear functions are what Aquila, our actual quantum computer, ingests. 

To declare a piecewise-linear waveform, we provide two lists: the first containing the durations of each linear segment, and another containing the values of the segments in their end points. The second list always contains one more entry than the first.

Let's see an example,

In [7]:
durations = [0.4,3.2,0.4]
values_MHz = [0.0,2.5,2.5,0.0]
values_radsec = [i * 2*np.pi for i in values_MHz]

waveform1 = piecewise_linear(durations, values_radsec)

waveform1.show()

An important fact that we should reiterate: all energetic parameters are *always* interpreted by Bloqade in units fo $rad/s$. However, numbers in $MHz$ units are usually simpler for user analysis. Thus, it is common practice to do as above, and create values in $MHz$ units, and then converting them to $rad/s$ by multiplying by adequate factors of $2\pi$.

Another waveform generating function that is relevant to know is `piecewise_constant()`. The way of operation is quite similar as above, but the number of elements in the durations and values lists must match. For example:

In [8]:
durations = [2.0,2.0]
values = [0.0,np.pi]

waveform1 = piecewise_constant(durations, values)

waveform1.show()

#### iv) Waveform hardware constraints

As with the geometric qubit placement choices, waveforms are also restricted by hardware capabilities. A global constraint on all time dependence: programs are limited to $4 \mu s$ (notice that coherence times, as measured by Rabi oscillations decay rates, for Aquila are estimated at above $6 \mu s$). Furthermore, the minimal time resolution for all waveform transformations is $0.05 \mu s$.

As for specific constraints, we have three distinct terms to take care of:

* $\Omega$ - The Rabi amplitude

The following constraints must be respected when defining waveforms for Rabi amplitudes:
1. the waveform must be defined in terms of piecewise-linear functions;
2. the wave form must begin and end at $0$;
3. the maximum value of $\Omega$ is $2.5 MHz$;
4. the maximum slew rate - speed at which we can ramp the waveform - is $40.0 MHz/\mu s$;

* $\Delta$ - The detuning

For the detuning $\Delta$, the following constraints must be respected:
1. the waveform must be defined in terms of piecewise-linear functions;
2. $\Delta$ must vary in the $-20 MHz$ to $+20 MHz$;
3. the maximum slew rate for $\Delta$ is $400.0 MHz/\mu s$


* $\phi$ - The phase

The phase $\phi$ must respect:
1. definition in terms of `piecewise_constant()` functions (in contrast with $\Delta$ and $\Omega$);
2. values determined in radians;


The plots below summarizes these points:




<img src="./assets/waveforms_summary.png" width="600" height="auto" />


In [ ]:
FreeResponseQuestion("exercise13b")